In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np

In [18]:
df_x = pd.read_csv("pre_processed_data/df_x_n_mfcc13.csv") 

In [19]:
df_y = pd.read_csv("pre_processed_data/df_y_n_mfcc13.csv")

In [20]:
x = np.array(df_x).reshape(-1, 13, 1)

In [21]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_y)

c:\Users\Yash Thakar\PROGRAMMING\Quantum\env\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
unique_elements, counts_elements = np.unique(y_encoded, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[  0   1   2   3   4]
 [174 229 210 222 138]]


In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, 
                                                    test_size=0.2, random_state = 42)

In [24]:
# Create the model
model = Sequential()

# Add convolutional layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(13, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax')) 


# Compile the model
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0002)
cb = [EarlyStopping(patience=10,monitor='accuracy',mode='max',restore_best_weights=True)]
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=16, validation_data=(x_test, y_test), callbacks = cb)

c:\Users\Yash Thakar\PROGRAMMING\Quantum\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.2120 - loss: 10.5832 - val_accuracy: 0.4256 - val_loss: 1.5837
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2440 - loss: 3.7604 - val_accuracy: 0.3385 - val_loss: 1.6050
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2381 - loss: 2.6084 - val_accuracy: 0.3949 - val_loss: 1.4731
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2587 - loss: 2.0971 - val_accuracy: 0.3897 - val_loss: 1.4665
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3362 - loss: 1.8306 - val_accuracy: 0.4513 - val_loss: 1.4073
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2864 - loss: 1.8103 - val_accuracy: 0.4256 - val_loss: 1.3716
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3122 - loss: 1.7210 - val_accuracy: 0.4974 - val_loss: 1.3417
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3063 - loss: 1.6278 - val_accuracy: 0.4923 -

In [25]:
y_pred = model.predict(x_test)
y_pred_classes = y_pred.argmax(axis=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [26]:
y_pred

array([[1.12535238e-01, 7.99975872e-01, 1.11328205e-02, 4.09071818e-02,
        3.54488604e-02],
       [1.98186740e-01, 1.33148670e-01, 2.48404965e-01, 2.26746097e-01,
        1.93513423e-01],
       [1.36377513e-01, 5.45880273e-02, 3.02173585e-01, 2.93308437e-01,
        2.13552386e-01],
       [1.85458601e-01, 1.40620559e-01, 2.63856739e-01, 2.07607299e-01,
        2.02456892e-01],
       [1.36080503e-01, 2.26698115e-01, 1.38093501e-01, 3.65159452e-01,
        1.33968413e-01],
       [1.73589483e-01, 7.45860487e-02, 2.12514788e-01, 3.73554051e-01,
        1.65755644e-01],
       [1.72483206e-01, 1.18116707e-01, 3.05036604e-01, 1.85582027e-01,
        2.18781471e-01],
       [5.61063825e-06, 9.99994159e-01, 1.24570898e-09, 1.45297264e-07,
        1.53976146e-07],
       [1.09642394e-01, 2.22235486e-01, 1.21633805e-01, 4.16843981e-01,
        1.29644364e-01],
       [3.40161860e-01, 2.28610456e-01, 1.35323554e-01, 1.49289370e-01,
        1.46614701e-01],
       [2.15665489e-01, 7.9488

In [27]:
y_pred_classes

array([1, 2, 2, 2, 3, 3, 2, 1, 3, 0, 2, 2, 1, 2, 1, 0, 1, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 4,
       2, 2, 3, 0, 0, 1, 2, 2, 2, 2, 2, 1, 1, 1, 3, 0, 3, 2, 3, 2, 3, 3,
       2, 1, 1, 1, 2, 2, 3, 2, 2, 3, 3, 1, 2, 2, 3, 1, 1, 2, 3, 3, 0, 3,
       1, 3, 2, 1, 2, 3, 1, 1, 3, 2, 0, 0, 2, 3, 1, 2, 3, 2, 3, 2, 1, 2,
       1, 3, 3, 4, 1, 2, 2, 3, 2, 2, 1, 2, 2, 3, 3, 2, 1, 2, 2, 3, 2, 1,
       3, 1, 3, 0, 2, 1, 2, 3, 2, 1, 1, 3, 1, 2, 3, 1, 2, 2, 1, 2, 2, 2,
       2, 3, 1, 1, 3, 1, 3, 2, 3, 3, 2, 1, 2, 3, 2, 1, 2, 3, 1, 3, 3, 2,
       3, 1, 1, 2, 3, 3, 2, 1, 2, 3, 3, 2, 2, 2, 1, 3, 2, 2, 2],
      dtype=int64)

In [28]:
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy Score: {accuracy}')

Accuracy Score: 0.5794871794871795


In [29]:
class_report = classification_report(y_test, y_pred_classes)
print('Classification Report:\n', class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.21      0.33        33
           1       0.98      0.80      0.88        56
           2       0.34      0.81      0.48        37
           3       0.58      0.64      0.61        45
           4       1.00      0.08      0.15        24

    accuracy                           0.58       195
   macro avg       0.74      0.51      0.49       195
weighted avg       0.73      0.58      0.56       195



In [30]:
y_pred = model.predict(x_train)
y_pred_classes = y_pred.argmax(axis=1)

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [31]:
accuracy = accuracy_score(y_train, y_pred_classes)
print(f'Accuracy Score: {accuracy}')

Accuracy Score: 0.5655526992287918


In [32]:
class_report = classification_report(y_train, y_pred_classes)
print('Classification Report:\n', class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.29      0.43       141
           1       0.98      0.75      0.85       173
           2       0.40      0.84      0.54       173
           3       0.52      0.66      0.58       177
           4       1.00      0.05      0.10       114

    accuracy                           0.57       778
   macro avg       0.74      0.52      0.50       778
weighted avg       0.72      0.57      0.54       778

